WEATHER API TESTING

In [2]:
import requests
from datetime import datetime

In [3]:
LAT,LON= 12.9716, 77.5946
LAT,LON

(12.9716, 77.5946)

In [8]:
unix_timestamp = int(datetime.now().timestamp())
unix_timestamp

1770195640

In [12]:
API_KEY = "5883b9ac08253fbd080c073b1832a6a8"

In [13]:
url = (
    "https://api.openweathermap.org/data/3.0/onecall/timemachine"
    f"?lat={LAT}&lon={LON}&dt={unix_timestamp}&appid={API_KEY}&units=imperial"
)
url

'https://api.openweathermap.org/data/3.0/onecall/timemachine?lat=12.9716&lon=77.5946&dt=1770195640&appid=5883b9ac08253fbd080c073b1832a6a8&units=imperial'

In [14]:
response = requests.get(url)
response.status_code

200

In [15]:
weather_data = []

if response.status_code == 200:
    data = response.json()
    today = datetime.now().date()

    day_weather = {
        "date": today,
        "temp": data["data"][0]["temp"],
        "humidity": data["data"][0]["humidity"],
        "weather_main": data["data"][0]["weather"][0]["main"],
        "weather_desc": data["data"][0]["weather"][0]["description"],
    }
    weather_data.append(day_weather)
else:
    print("Failed to fetch data for today")

weather_data

[{'date': datetime.date(2026, 2, 4),
  'temp': 81.27,
  'humidity': 31,
  'weather_main': 'Clear',
  'weather_desc': 'clear sky'}]

CONNECTING TO MYSQL

In [16]:
pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 36.2 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip available: 22.3.1 -> 26.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [21]:

import getpass
import mysql.connector

In [25]:
host = "localhost"
port = 3306
database = "retail_db"

user = input("Enter MySQL username: ")
password = getpass.getpass("Enter MySQL password: ")

conn = mysql.connector.connect(
    host=host,
    port=port,
    user=user,
    password=password,
    database=database
)

print("Connected successfully.")


Connected successfully.


In [26]:
def run_query(sql: str, params=None, max_rows: int = 20):
    cur = conn.cursor()
    cur.execute(sql, params or ())
    rows = cur.fetchmany(max_rows)
    cols = [d[0] for d in cur.description] if cur.description else []
    cur.close()
    return cols, rows

In [28]:
cols, rows = run_query("SELECT * FROM orders LIMIT 10;")

print(cols)
for r in rows:
    print(r)

['order_id', 'order_date', 'order_customer_id', 'order_status']
(1, datetime.datetime(2013, 7, 25, 0, 0), 11599, 'CLOSED')
(2, datetime.datetime(2013, 7, 25, 0, 0), 256, 'PENDING_PAYMENT')
(3, datetime.datetime(2013, 7, 25, 0, 0), 12111, 'COMPLETE')
(4, datetime.datetime(2013, 7, 25, 0, 0), 8827, 'CLOSED')
(5, datetime.datetime(2013, 7, 25, 0, 0), 11318, 'COMPLETE')
(6, datetime.datetime(2013, 7, 25, 0, 0), 7130, 'COMPLETE')
(7, datetime.datetime(2013, 7, 25, 0, 0), 4530, 'COMPLETE')
(8, datetime.datetime(2013, 7, 25, 0, 0), 2911, 'PROCESSING')
(9, datetime.datetime(2013, 7, 25, 0, 0), 5657, 'PENDING_PAYMENT')
(10, datetime.datetime(2013, 7, 25, 0, 0), 5648, 'PENDING_PAYMENT')


In [29]:
cols, rows = run_query("SELECT COUNT(*) AS total_orders FROM orders;")
print(cols)
print(rows[0])

['total_orders']
(68883,)


In [30]:
cols, rows = run_query("DESC orders;", max_rows=200)

print(cols)
for r in rows:
    print(r)

['Field', 'Type', 'Null', 'Key', 'Default', 'Extra']
('order_id', 'int', 'NO', 'PRI', None, 'auto_increment')
('order_date', 'datetime', 'NO', '', None, '')
('order_customer_id', 'int', 'NO', 'MUL', None, '')
('order_status', 'varchar(45)', 'NO', 'MUL', None, '')


In [31]:
conn.close()
print("Connection closed.")

Connection closed.
